<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# Комментарий автора

В связи с тем, что данный проект я делаю уже после дэдлайна, то мне приходится срезать углы на качестве и наполнении.

Так же, в связи с огромным дата сетом - обучение очень долгое, поэтому и кол-во моделей я урезал. Я даже оставлял на ночь прогонять варианты, но не получил результата. 

"Прошу понять и простить" (с) Бородач

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from nltk.corpus import stopwords 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from sklearn.model_selection import cross_val_score

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
RANDOM_STATE = 78910

**Анализ исходных данных** 

In [4]:
data = pd.read_csv('/datasets/toxic_comments.csv', index_col = 0)

In [5]:
display(data.head(3))

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0


In [6]:
display(data.tail(3))

,text,toxic
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0
159450,"""\nAnd ... I really don't think you understand...",0


In [7]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


None

In [8]:
print("Кол-во дупликатов:",data.duplicated().sum())

Кол-во дупликатов: 0


In [9]:
print("Кол-во пропусков:")
display(data.isna().sum())

Кол-во пропусков:


text     0
toxic    0
dtype: int64

**Обработка данных для обучения** 

In [10]:
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [11]:
lemmatizer = WordNetLemmatizer()

In [12]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemm(text):
    text = re.sub(r"[^a-zA-z ]"," ", text.lower())
    token = nltk.word_tokenize(text)
    text = [word for word in token if word not in stop_words] 
    text = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in text]
    text = " ".join(text)
    return text


In [13]:
%%time

data["lemmatized"] = data["text"].apply(lemm)

CPU times: user 11min 9s, sys: 55.6 s, total: 12min 5s
Wall time: 12min 6s


In [14]:
display(data.head(3))

,text,toxic,lemmatized
0,Explanation\nWhy the edits made under my usern...,0,explanation edits make username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,aww match background colour seemingly stuck th...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man really try edit war guy constantly rem...


In [15]:
display(data.tail(3))

,text,toxic,lemmatized
159448,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm there actual article prostitution ...
159449,And it looks like it was actually you who put ...,0,look like actually put speedy first version de...
159450,"""\nAnd ... I really don't think you understand...",0,really think understand come idea bad right aw...


In [16]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    data, data["toxic"], test_size=0.2, random_state=RANDOM_STATE, stratify = data["toxic"])

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=RANDOM_STATE, stratify = target_valid_test)

In [17]:
print("Размер features_test:",features_test.shape)
print("Размер features_valid:",features_valid.shape)
print("Размер features_train:",features_train.shape)
print("Размер target_test:",target_test.shape)
print("Размер target_valid:",target_valid.shape)
print("Размер target_train:",target_train.shape)

Размер features_test: (15930, 3)
Размер features_valid: (15929, 3)
Размер features_train: (127433, 3)
Размер target_test: (15930,)
Размер target_valid: (15929,)
Размер target_train: (127433,)


In [18]:
features_train_corpus = features_train["lemmatized"]
features_valid_corpus = features_valid["lemmatized"]
features_test_corpus = features_test["lemmatized"]

In [19]:
nltk.download("stopwords")
stopwords = set(nltk_stopwords.words("english"))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(features_train_corpus)
tf_idf_valid = count_tf_idf.transform(features_valid_corpus)
tf_idf_test = count_tf_idf.transform(features_test_corpus)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
print("Размер матрицы:", tf_idf_train.shape)
print("Размер матрицы:", tf_idf_valid.shape)
print("Размер матрицы:", tf_idf_test.shape)

Размер матрицы: (127433, 135967)
Размер матрицы: (15929, 135967)
Размер матрицы: (15930, 135967)


## Обучение

In [21]:
%%time

rf_g_model = RandomForestClassifier(criterion = "gini",random_state = RANDOM_STATE)

rf_g_model.fit(tf_idf_train,target_train)

print("F1 score:",f1_score(target_valid,rf_g_model.predict(tf_idf_valid)))

F1 score: 0.6977281478629188
CPU times: user 9min 44s, sys: 3.53 s, total: 9min 48s
Wall time: 9min 48s


In [22]:
%%time

rf_e_model = RandomForestClassifier(criterion = "entropy",random_state = RANDOM_STATE)

rf_e_model.fit(tf_idf_train,target_train)

print("F1 score:",f1_score(target_valid,rf_e_model.predict(tf_idf_valid)))

F1 score: 0.6578313253012049
CPU times: user 10min 16s, sys: 3.31 s, total: 10min 20s
Wall time: 10min 21s


In [23]:
%%time

lr_model = LogisticRegression(random_state = RANDOM_STATE)

tuned_parameters = {"penalty": ["l1","l2","elasticnet","none"]}

lr_model = GridSearchCV(estimator = lr_model,
                     param_grid = tuned_parameters,
                     n_jobs = -1,
                     cv = 5,
                     scoring='f1')

lr_model.fit(tf_idf_train,target_train)

print(lr_model.best_params_)
print(lr_model.best_score_)

{'penalty': 'none'}
0.7259466296912915
CPU times: user 2min 9s, sys: 5min 3s, total: 7min 13s
Wall time: 7min 13s


In [24]:
%%time

cb_s_model = CatBoostClassifier(grow_policy = "SymmetricTree",logging_level = "Silent",random_state = RANDOM_STATE)

cb_s_model.fit(tf_idf_train,target_train)

print("F1 score:",f1_score(target_valid,cb_s_model.predict(tf_idf_valid)))

F1 score: 0.7565813198701767
CPU times: user 35min 21s, sys: 27 s, total: 35min 48s
Wall time: 35min 51s


**Вывод:**

Лучшей моделью оказалось модель cb_s_model, а именно CatBoostClassifier с политикой роста "SymmetricTree". F1 метрика на тренировочной выборке показала 0.75

Результаты RandomForestClassifier показали самый низкий результат с F1 метрикой в среднем 0.67. 

## Тестирование

Проверим модели на тестовой выборке: 

In [25]:
%%time 

predictions = rf_g_model.predict(tf_idf_test)

print(f1_score(target_test,predictions))

0.7062857142857143
CPU times: user 2.37 s, sys: 24.9 ms, total: 2.4 s
Wall time: 2.41 s


In [26]:
%%time 

predictions = rf_e_model.predict(tf_idf_test)

print(f1_score(target_test,predictions))

0.6594115276098348
CPU times: user 1.96 s, sys: 23.7 ms, total: 1.99 s
Wall time: 2 s


In [27]:
%%time 

predictions = lr_model.predict(tf_idf_test)

print(f1_score(target_test,predictions))

0.7563938618925831
CPU times: user 8.08 ms, sys: 190 µs, total: 8.27 ms
Wall time: 6.99 ms


In [28]:
%%time 

predictions = cb_s_model.predict(tf_idf_test)

print(f1_score(target_test,predictions))

0.7733050847457628
CPU times: user 290 ms, sys: 7.84 ms, total: 298 ms
Wall time: 307 ms


## Выводы

Как и ожидалось, RandomForestClassifier не показал хорошего результата. Поэтому, стоит сконцентрировать внимание на модели cb_s_model, которая показала метрику F1 равной 0.77 на тестовой выборке, чем подтвердила свою работоспособность. Однако она заняла больше всего времени на обучение и на предсказание, в разы больше чем другие модели. 